In [2]:
import torch
import numpy as np
from enum import Enum


class SimulatedBifurcationEngine:
    """Simplified implementation of the Simulated Bifurcation engine variants."""
    
    @staticmethod
    def ballistic(x):
        """Ballistic SB activation function (identity)."""
        return x
    
    @staticmethod
    def discrete(x):
        """Discrete SB activation function (sign)."""
        return torch.sign(x)


class SymplecticIntegrator:
    """Simulates the evolution of spins' momentum and position."""
    
    def __init__(self, shape, activation_function, dtype=torch.float32, device="cpu"):
        """Initialize the integrator with random positions and momenta."""
        self.position = 2 * torch.rand(size=shape, device=device, dtype=dtype) - 1
        self.momentum = 2 * torch.rand(size=shape, device=device, dtype=dtype) - 1
        self.activation_function = activation_function
    
    def position_update(self, coefficient):
        """Update position based on momentum."""
        self.position += coefficient * self.momentum
    
    def momentum_update(self, coefficient):
        """Update momentum based on position."""
        self.momentum += coefficient * self.position
    
    def quadratic_momentum_update(self, coefficient, matrix):
        """Update momentum based on quadratic interaction."""
        self.momentum = self.momentum + coefficient * matrix @ self.activation_function(self.position)
    
    def simulate_inelastic_walls(self):
        """Enforce boundary conditions on position."""
        self.momentum[torch.abs(self.position) > 1.0] = 0.0
        self.position = torch.clamp(self.position, -1.0, 1.0)
    
    def step(self, momentum_coefficient, position_coefficient, quadratic_coefficient, matrix):
        """Perform one step of the symplectic integration."""
        self.momentum_update(momentum_coefficient)
        self.position_update(position_coefficient)
        self.quadratic_momentum_update(quadratic_coefficient, matrix)
        self.simulate_inelastic_walls()
    
    def sample_spins(self):
        """Convert positions to spin values."""
        return torch.where(self.position >= 0.0, 1.0, -1.0)


class OptimizationDomain(Enum):
    """Possible domains for optimization variables."""
    SPIN = 1
    BINARY = 2
    INTEGER = 3


class SimplifiedMarkowitz:
    """A simplified implementation of the Markowitz portfolio optimization model."""
    
    def __init__(
        self,
        covariance,
        expected_return,
        risk_coefficient=1,
        number_of_bits=1,
        dtype=torch.float32,
        device="cpu"
    ):
        """
        Initialize the Markowitz portfolio optimization model.
        
        Parameters:
        -----------
        covariance : torch.Tensor
            Covariance matrix of asset returns
        expected_return : torch.Tensor
            Expected returns vector
        risk_coefficient : float
            Risk aversion coefficient (higher means more risk-averse)
        number_of_bits : int
            Number of bits to represent each asset quantity
        dtype : torch.dtype
            Data type for computations
        device : str or torch.device
            Device to perform computations on
        """
        # Store input parameters
        self.covariance = self._ensure_tensor(covariance, dtype, device)
        self.expected_return = self._ensure_tensor(expected_return, dtype, device)
        self.risk_coefficient = risk_coefficient
        self.number_of_bits = number_of_bits
        self.dtype = dtype
        self.device = device
        
        # Calculate dimensions
        self.assets = self.covariance.shape[0]
        
        # Initialize result storage
        self.portfolio = None
        
        # Prepare Ising model coefficients
        self.matrix, self.vector = self._prepare_optimization_coefficients()
    
    def _ensure_tensor(self, data, dtype, device):
        """Convert input data to tensor if necessary."""
        if isinstance(data, torch.Tensor):
            return data.to(dtype=dtype, device=device)
        else:
            return torch.tensor(data, dtype=dtype, device=device)
    
    def _prepare_optimization_coefficients(self):
        """Prepare the coefficient matrix and vector for optimization."""
        # For Markowitz, the quadratic term is -0.5 * risk_coefficient * covariance
        matrix = -0.5 * self.risk_coefficient * self.covariance
        
        # The linear term is expected_return
        vector = self.expected_return
        
        return matrix, vector
    
    def _binary_to_integer(self, binary_solution):
        """Convert binary spin solution to integer weights."""
        # Calculate the integer values (number of assets)
        max_val = 2**self.number_of_bits - 1
        weights = ((binary_solution + 1) / 2) * max_val
        return weights
    
    def _run_simulated_bifurcation(
        self,
        agents=128,
        max_steps=1000,
        time_step=0.05,
        mode="ballistic",
        heated=False,
        verbose=True
    ):
        """Run the Simulated Bifurcation algorithm to find optimal portfolio."""
        # Setup
        activation_fn = SimulatedBifurcationEngine.ballistic if mode == "ballistic" else SimulatedBifurcationEngine.discrete
        integrator = SymplecticIntegrator((self.assets, agents), activation_fn, self.dtype, self.device)
        
        # Calculate scaling parameter for stability
        scaling = 0.5 * (self.assets - 1)**0.5 / torch.sqrt(torch.sum(self.matrix**2))
        
        # Initialize pressure parameter
        pressure_slope = 0.01
        
        # Run symplectic integration
        if verbose:
            print("Starting Simulated Bifurcation optimization...")
        
        for step in range(max_steps):
            # Calculate current pressure (annealing parameter)
            pressure = min(time_step * step * pressure_slope, 1.0)
            
            # Calculate coefficients
            momentum_coef = time_step * (pressure - 1.0)
            position_coef = time_step
            quadratic_coef = time_step * scaling
            
            # Perform integration step
            integrator.step(momentum_coef, position_coef, quadratic_coef, self.matrix)
            
            # Optional progress report
            if verbose and step % 100 == 0:
                print(f"Step {step}/{max_steps}")
        
        # Sample final spins
        spins = integrator.sample_spins()
        return spins
    
    def _convert_spins_to_portfolio(self, spins):
        """Convert spin variables to portfolio weights."""
        # Convert to integer representation
        weights = self._binary_to_integer(spins)
        return weights
    
    def _calculate_objective(self, weights):
        """Calculate the objective function value for given weights."""
        expected_gain = weights @ self.expected_return
        risk = 0.5 * self.risk_coefficient * weights @ self.covariance @ weights
        return expected_gain - risk
    
    def maximize(
        self,
        agents=128,
        max_steps=1000,
        mode="ballistic",
        heated=False,
        verbose=True
    ):
        """
        Maximize the portfolio returns using Simulated Bifurcation.
        
        Parameters:
        -----------
        agents : int
            Number of parallel optimization agents
        max_steps : int
            Maximum number of simulation steps
        mode : str
            'ballistic' or 'discrete' SB algorithm variant
        heated : bool
            Whether to use heated SB variant
        verbose : bool
            Whether to print progress information
        
        Returns:
        --------
        portfolio : torch.Tensor
            Optimized portfolio weights
        gain : float
            Expected gain of the optimized portfolio
        """
        # Run SB algorithm
        spins = self._run_simulated_bifurcation(
            agents=agents,
            max_steps=max_steps,
            mode=mode,
            heated=heated,
            verbose=verbose
        )
        
        # Convert all solutions to portfolios
        all_portfolios = self._convert_spins_to_portfolio(spins)
        
        # Evaluate all solutions
        gains = torch.zeros(agents, device=self.device, dtype=self.dtype)
        for i in range(agents):
            weights = all_portfolios[:, i]
            gains[i] = self._calculate_objective(weights)
        
        # Find best solution
        best_idx = torch.argmax(gains)
        self.portfolio = all_portfolios[:, best_idx]
        self.gains = gains[best_idx].item()
        
        return self.portfolio, self.gains


# For backward compatibility with the original test
def Markowitz(*args, **kwargs):
    """Factory function for SimplifiedMarkowitz."""
    return SimplifiedMarkowitz(*args, **kwargs)


# Example usage
if __name__ == "__main__":
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    # Create test data
    covariance = torch.tensor([[1.0, 1.2, 0.7], 
                              [1.2, 1.0, -1.9], 
                              [0.7, -1.9, 1.0]])
    expected_return = torch.tensor([0.2, 0.05, 0.17])
    
    # Create and run model
    model = SimplifiedMarkowitz(
        covariance,
        expected_return,
        risk_coefficient=1,
        number_of_bits=3,
        dtype=torch.float32
    )
    
    portfolio, gain = model.maximize(agents=10, max_steps=5000, verbose=True)
    
    print("\nOptimized Portfolio:")
    print(f"Weights: {portfolio}")
    print(f"Expected Gain: {gain:.4f}")




Starting Simulated Bifurcation optimization...
Step 0/5000
Step 100/5000
Step 200/5000
Step 300/5000
Step 400/5000
Step 500/5000
Step 600/5000
Step 700/5000
Step 800/5000
Step 900/5000
Step 1000/5000
Step 1100/5000
Step 1200/5000
Step 1300/5000
Step 1400/5000
Step 1500/5000
Step 1600/5000
Step 1700/5000
Step 1800/5000
Step 1900/5000
Step 2000/5000
Step 2100/5000
Step 2200/5000
Step 2300/5000
Step 2400/5000
Step 2500/5000
Step 2600/5000
Step 2700/5000
Step 2800/5000
Step 2900/5000
Step 3000/5000
Step 3100/5000
Step 3200/5000
Step 3300/5000
Step 3400/5000
Step 3500/5000
Step 3600/5000
Step 3700/5000
Step 3800/5000
Step 3900/5000
Step 4000/5000
Step 4100/5000
Step 4200/5000
Step 4300/5000
Step 4400/5000
Step 4500/5000
Step 4600/5000
Step 4700/5000
Step 4800/5000
Step 4900/5000

Optimized Portfolio:
Weights: tensor([0., 7., 7.])
Expected Gain: 45.6400


In [ ]:

model.maximize(agents=10, verbose=False)
assert torch.equal(torch.tensor([0.0, 7.0, 7.0]), model.portfolio)
assert 45.64 == round(model.gains, 2)
